In [1]:
import spacy
import scispacy
import pandas as pd

from scispacy.linking import EntityLinker

In [2]:
BLENDER_REL_FILE_LISTS = ["genes_diseases_relation.csv","chem_gene_ixns_relation.csv", "chemicals_diseases_relation.csv" ]

PATH_TO_DATA = "../../../for_edan/"


In [6]:
nlp = spacy.load("en_core_sci_sm")
# linker = EntityLinker(resolve_abbreviations=True, name="umls")
config = {
    "linker_name": "umls",
    "resolve_abbreviations": True
}
linker = nlp.add_pipe("scispacy_linker", config = config)

In [12]:

"""
Input: tui_filename (file name with tag to category name)
Create dict of tag to category name based on file
"""
def read_type_ids(tui_filename):
  tui_label_dict = {}
  input_file = open(tui_filename)
  for line in input_file:
    line_parts = line[:-1].split("\t")
    tui_label_dict[line_parts[0]] = line_parts[1]
  return tui_label_dict

"""
Input: file_path (file name)
copy lines of file to elements of list but remove newline
"""
def read_temp_file(file_path):
  input_file = open(file_path)
  res = []
  for line in input_file:
    res.append(line[:-1])
  return res

"""
Depricated
"""
def write_cofie_entities(cofie_kb_path):
    seen_spans = []
    output_file = open(PATH_TO_DATA + "cofie_spans.txt", "w")
    input_file = open(cofie_kb_path)
    count = 0
    for line in input_file:

      if count % 10000 == 0:
        print(count)
      count += 1
      
      if count == 1:
        continue
      line_parts = line[:-1].split("\t")
      # import pdb; pdb.set_trace()
      for i in [2,3]:
        if line_parts[i] not in seen_spans:
          output_file.write(line_parts[i] + '\n')
          seen_spans.append(line_parts[i])
          
"""
Depricated
"""
def write_blender_entities(blender_kb_path):
    # seen_spans = []
    seen_spans = read_temp_file("blender_spans_temp.txt")
    output_file = open("blender_spans.txt", "w")
    for span in seen_spans:
      output_file.write(span + '\n')
    for file in BLENDER_REL_FILE_LISTS:
      df = pd.read_csv(blender_kb_path + file, header=0, sep="\t")

      for row in df.iterrows():
        if file == "genes_diseases_relation.csv" and int(row[0]) < 43700000:
          continue
        if row[0] % 100000 == 0:
          print(file)
          print(str(row[0]))
        if file == "genes_diseases_relation.csv":
          span1 = str(row[1]["GeneSymbol"]).lower()
          span2 = str(row[1]["DiseaseName"]).lower()
        elif file == "chem_gene_ixns_relation.csv":
          span1 = str(row[1]["ChemicalName"]).lower()
          span2 = str(row[1]["GeneName"]).lower()
        else:
          span1 = str(row[1]["ChemicalName"]).lower()
          span2 = str(row[1]["DiseaseName"]).lower()

        if span1 not in seen_spans:
          seen_spans.append(span1)
          output_file.write(span1 + '\n')

        if span2 not in seen_spans:
          seen_spans.append(span2)
          output_file.write(span2 + '\n')

 
"""
Input: input_filename spans extracted from KB
       output_filename TODO figure out
       
"""
def get_types(input_filename, output_filename):
    type_count_dict = {}
    log_file = open(PATH_TO_DATA + "training_spans_subcats.tsv", "w")
    output_file = open(output_filename, "w")
    # This line takes a while, because we have to download ~1GB of data
    # and load a large JSON file (the knowledge base). Be patient!
    # Thankfully it should be faster after the first time you use it, because
    # the downloads are cached.
    # NOTE: The resolve_abbreviations parameter is optional, and requires that
    # the AbbreviationDetector pipe has already been added to the pipeline. Adding
    # the AbbreviationDetector pipe and setting resolve_abbreviations to True means
    # that linking will only be performed on the long form of abbreviations.
    tui_label_dict = read_type_ids(PATH_TO_DATA + "tui_labels.tsv")
    input_file = open(input_filename)
    count = 0
    for line in input_file:
      count += 1
      if count % 1000 == 0:
        print("count " + str(count))

      # extract key entities
      doc = nlp(line[:-1])
      if len(doc.ents) != 0:
        for i in range(len(doc.ents)):
          entity = doc.ents[i]
          for umls_ent in entity._.kb_ents:
            types_list = linker.kb.cui_to_entity[umls_ent[0]].types
            # Count occurences of each TUI in dictionary
            for _type in types_list:
              if _type not in type_count_dict:
                type_count_dict[_type] = 0
              type_count_dict[_type] += 1
              # import pdb; pdb.set_trace()
              log_file.write(line[:-1] + "\t" + tui_label_dict[_type]+ "\n")
    output_file = open(output_filename, "w")
    for _type in type_count_dict:
      output_file.write(_type + "\t" + str(type_count_dict[_type]) + '\n')

      

# write_cofie_entities("/home/aida/covid_clean/dygiepp/complete_KB_coref.tsv")
# write_blender_entities("/data/aida/")
#get_types("blender_spans2.txt", "blender_spans_tags.txt")
doc = get_types(PATH_TO_DATA + "training_spans.txt", PATH_TO_DATA + "training_type_count.tsv")





count 1000
count 2000
count 3000


In [8]:
dir(linker.kb)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'alias_to_cuis',
 'cui_to_entity',
 'semantic_type_tree']

In [80]:
print(doc.ents[0]._.umls_ents)
print(dir(doc.ents[0]._))

[]
['get', 'has', 'kb_ents', 'set', 'umls_ents']


In [31]:
next(input_file)

'long - term imaging follow - up after flow - diverter implantation\n'

In [76]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily")


In [85]:
doc.ents[0].vector

array([ 1.0921346 , -0.24784482,  0.7017714 ,  0.3013472 ,  0.04505855,
       -1.0176959 ,  0.7290837 , -1.083273  , -0.80804557, -0.1108322 ,
       -0.46059218,  1.1197779 ,  1.5859103 , -1.5850055 , -0.19468474,
        0.86162543, -1.1898143 ,  0.4025337 ,  0.546126  ,  0.3930942 ,
       -0.5489894 ,  0.07679972, -1.3166326 , -0.6879619 ,  0.7433906 ,
        0.6149941 ,  0.152344  , -0.32041124, -0.47136348,  0.50137335,
        0.11929529, -0.28746825,  0.21233952, -1.5002258 , -0.53008753,
       -0.5166469 , -0.98063946, -1.1089299 ,  1.5003526 , -0.0623616 ,
       -0.63929164, -0.12292683, -0.16088364, -0.6792747 ,  0.1937968 ,
        0.83047587,  1.5787429 ,  0.87463486, -0.15952143,  0.85969055,
        0.36707333, -0.29322675, -0.29202163, -0.17316845, -0.82743335,
        0.32459146,  0.9104742 , -0.12151811,  0.40474847,  0.43918195,
        1.6888247 , -1.3032068 ,  0.97610736, -0.7258777 ,  0.10229971,
        0.1360836 , -1.1312985 , -0.7466174 ,  0.78176427,  1.49

In [78]:
for ent in doc.ents:
    for umls_ent in ent._.kb_ents:
        print(umls_ent)

('C0521329', 1.0)
('C0037922', 0.8068786859512329)
('C3887662', 0.7646855711936951)
('C1334264', 0.7219825983047485)
('C0037925', 0.7107230424880981)
('C1947952', 1.0)
('C0032372', 0.7856597900390625)
('C2586323', 0.7522947192192078)
('C1744560', 0.7451043128967285)
('C0030442', 0.723750114440918)
('C0026846', 1.0)
('C0541794', 1.0)
('C0026847', 0.8910943865776062)
('C1848736', 0.884716272354126)
('C0043116', 0.8696750998497009)
('C1705240', 0.9999998807907104)
('C1839259', 0.9999998807907104)
('C0439660', 1.0)
('C0019247', 0.8003369569778442)
('C4277541', 0.7369471192359924)
('C4277511', 0.735810399055481)
('C0598589', 0.7073548436164856)
('C0085084', 1.0)
('C1865412', 0.8891978859901428)
('C0007595', 0.8664658665657043)
('C4761515', 0.8664658665657043)
('C1654621', 0.8529700636863708)
('C0196940', 0.8281357884407043)
('C1516670', 0.8198980689048767)
('C0032500', 0.7472081780433655)
('C0392213', 0.7402303218841553)
('C0034786', 1.0)
('C1367578', 1.0)
('C1447749', 1.0)
('C4764257', 0.8

In [24]:
nlp = spacy.load("en_core_sci_lg")

In [47]:
nlp = spacy.load("en_core_sci_sm")

In [48]:
nlp("Organism").ents[0].similarity(nlp("Organism"))

1.0

In [49]:
nlp("Organism").ents[0].similarity(nlp("Animal").ents[0])

/home/edan/miniconda3/envs/spacy/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """Entry point for launching an IPython kernel.


0.719016

In [51]:
tui_label_dict = read_type_ids(PATH_TO_DATA + "tui_labels.tsv")
categories = ["Animal", "Organism", "Drug", "Cell", "Behavior", "Organization",
              "Group", "Disease", "Symptom", "Body Part", "Organ", "Medical Device",
              "Product", "Concept", "Finding", "Procedure", "Molecule", "Organic Chemical", "Virus"]

output_file = open("categories_sm.tsv", "w")
def get_cat(doc):
    top_cat = "None"
    top_prob = 0
    for cat in categories:
        doc2 = nlp(cat)
        prob = doc.similarity(doc2)
        if (top_prob < prob):
            top_cat = cat
            top_prob = prob
    assert top_cat != "None"
    return top_cat, top_prob
    

for value in tui_label_dict.values():
    doc = nlp(value)
    cat, prob = get_cat(doc)
    output_file.write(value + "\t" + cat + "\t" + str(prob) + "\n")
    
    

/home/edan/miniconda3/envs/spacy/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  if sys.path[0] == '':


In [72]:
doc.similarity(doc2)

0.23025123493015764

In [58]:

doc.vector.dot(doc2.vector) / doc.vector_norm / doc2.vector_norm

0.1925040384439355

In [57]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_get_array_attrs',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set_ents',
 'se